In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# 假设你的数据是一个Series
df_good = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements_with_stability.pkl')
df_py = df_good[df_good['buzaki_py_cell_type']=='pyramidal']
df = df_py
base_folder = r"Q:/sachuriga/CR_CA1_paper/Results/functional_cell_type"
control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']
sessions = ["A"]


for session in sessions:
    # Filter for sessions
    if session == "Total":
        df_a = df
    else:
        df_a = df[df['session'] == session]

    # Separate into control and experimental groups
    control_df = df_a[df_a['animal_id'].isin(control_ids)]
    exp_df = df_a[df_a['animal_id'].isin(exp_ids)]


df_con_deep = control_df[control_df['sub_population']=="deep"].reset_index(drop=True)['Information_content_rate']
df_con_superficial = control_df[control_df['sub_population']=="superficial"].reset_index(drop=True)['Information_content_rate']
df_exp_deep = exp_df[exp_df['sub_population']=="deep"].reset_index(drop=True)['Information_content_rate']
df_exp_superficial = exp_df[exp_df['sub_population']=="superficial"].reset_index(drop=True)['Information_content_rate']

def group_indices_by_step(df_col, step=0.075):
    used_indices = set()
    result = []
    values = df_col.copy()
    max_val = values.max()

    while len(used_indices) < len(values):
        remaining = values[~values.index.isin(used_indices)]
        if remaining.empty:
            break

        current_group = []
        current_val = remaining.min()
        current_idx = remaining.idxmin()
        current_group.append(current_idx)
        used_indices.add(current_idx)

        while True:
            target_val = current_val + step
            remaining = values[~values.index.isin(used_indices)]
            if remaining.empty:
                break

            # 找到大于等于 target_val 的值中最接近 target_val 的那个
            diffs = remaining - target_val
            diffs = diffs[diffs >= 0]
            if diffs.empty:
                break

            next_idx = diffs.idxmin()
            current_val = values[next_idx]
            current_group.append(next_idx)
            used_indices.add(next_idx)

        result.append(current_group)

    return result
results_con_deep = group_indices_by_step(df_con_deep)
results_con_superficial = group_indices_by_step(df_con_superficial)
results_exp_deep = group_indices_by_step(df_exp_deep)
results_exp_superficial = group_indices_by_step(df_exp_superficial)

def plot_rate_map_panel(ax, results_con, df, sublayer="deep", group = "control", map_color='jet'):
    i_row = 0
    base_y = 0.35
    y_step = 0.5

    for indices in results_con:
        y_coord = base_y + (i_row * y_step)

        if len(indices) > 0:
            values = [df[i] for i in indices]
            num_ratemaps = len(indices)
            for i, (idx, value) in enumerate(zip(indices, values)):
                img_path = fr"Q:\sachuriga\CR_CA1_paper\Results\rate map histogram\{group}/{sublayer}/{map_color}/{idx}.png"
                plot_image_at_xy(ax, img_path, value, y_coord, max_size=0.014)
        i_row += 1
    ax.set_xlim(0, 4.5)
    ax.set_ylim(0, 3.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(False)
    ax.tick_params(axis='x', labelsize=10.5)
    ax.set_xlabel("Information Content Rate (spikes / bit)", fontsize=10.5)
    ax.set_yticks([])
    ax.vlines(x=[1, 2, 3, 4], ymin=0, ymax=8.5, colors='grey', linestyles='dashed', linewidth=3)
    
# Function to load and plot an image with dynamic size adjustment
def plot_image_at_xy(ax, img_path, x, y, max_size=0.09):
    if os.path.exists(img_path):
        img = plt.imread(img_path)
        img_height, img_width = img.shape[:2]
        aspect_ratio = img_width / img_height

        # Calculate zoom based on max_size and aspect ratio
        trans = ax.transData
        fig = ax.get_figure()
        dpi = fig.dpi
        xlim = ax.get_xlim()
        data_width = xlim[1] - xlim[0]
        bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        axes_width_pixels = bbox.width * dpi
        pixels_per_data = axes_width_pixels / data_width
        size_pixels = max_size * pixels_per_data
        # zoom = size_pixels / max(img_width, img_height / aspect_ratio)

        # imagebox = OffsetImage(img, zoom=zoom)
        # ab = AnnotationBbox(imagebox, (x, y), frameon=False, pad=0)
        # ax.add_artist(ab)


        zoom = 0.017  # Adjust this value to control ratemap size
        imagebox = OffsetImage(img, zoom=zoom)
        ab = AnnotationBbox(imagebox, (x, y), frameon=False, pad=0)
        ax.add_artist(ab)
        return ab
    else:
        print(f"Image {img_path} not found")
        return None

import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp, shapiro, ttest_ind, mannwhitneyu
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np

# Data loading
folder_path = r"S:\Sachuriga\file_with_table\ripple_ch"

def get_pkl_files(folder_path):
    all_files = os.listdir(folder_path)
    pkl_files = [f for f in all_files if f.endswith("withDLC.pkl")]
    return pkl_files

target_prefixes_control = ['65165', '65091', '63383', '66539', '65622']
target_prefixes_exp = ['65588', '63385', '66538', '66537', '66922']
pkl_files = get_pkl_files(folder_path)

# # Variables to analyze
# variables = ['Information_content_rate', 'Sparsity', 'Field_size', 'Averate_rate', 'bursting_index', 'Selectivity']
# titles = ['Information content rate\n (spikes/bit)', 'Sparsity', 'Max Field Size', 'Firing Rate (Hz)', 'Bursting iondex', 'Selectivity']


variables = ['amplitude_median','half_width','peak_to_valley','peak_trough_ratio', 'recovery_slope', 'repolarization_slope']
titles = ['Amplitude median (mV)', 'Half width (ms)', 'Peak to valley (ms)', 
            'peak trough ratio', 'Recovery slope', 'repolarization_slope']
        
all_dfs = []
scale_transefers = [0.001,1000,1000,1,0.00001,0.000001]


# Process each pickle file
for file in pkl_files:
    try:
        df = pd.read_pickle(os.path.join(folder_path, file))
    except Exception as e:
        continue
    #df = df[(df['cell_type'] == "pyramidal") & (df['session'] == "A")]

    df = df[(df['buzaki_py_cell_type'] == "pyramidal") & (df['session'] == "A")  & (df['unit_quality'] == "good")]
    #df = df[df['session'] == "A"]
    if df.empty:
        print(f"Warning: File {file} has no pyramidal cells")
        continue
    try:
        animal_id = df['animal_id'].iloc[0]
    except KeyError:
        print(f"Warning: File {file} does not have 'animal_id' column")
        continue
    if any(animal_id.startswith(prefix) for prefix in target_prefixes_control):
        df['group_ani'] = 'control'
    elif any(animal_id.startswith(prefix) for prefix in target_prefixes_exp):
        df['group_ani'] = 'exp'
    else:
        print(f"Warning: animal_id {animal_id} does not match any group")
        continue
    all_dfs.append(df)

y_pos = "addjust y r2"
combined_df=[]
combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df['depth'] = combined_df[y_pos].apply(lambda x: 'deep' if x > 0 else 'superficial')
combined_df['group_depth'] = combined_df['group_ani'] + '_' + combined_df['depth']

# Convert variables to numeric and handle invalid values
for var in variables:
    combined_df[var] = pd.to_numeric(combined_df[var], errors='coerce')
    if combined_df[var].isna().any():
        print(f"Warning: {var} contains NaN values after conversion to numeric")

# Function to test normality and choose test
def choose_stat_test(data1, data2, var_name, group1_name, group2_name):
    # Drop NaN values and ensure numeric
    data1 = data1.dropna()
    data2 = data2.dropna()
    
    # Check for non-numeric values
    if not np.issubdtype(data1.dtype, np.number) or not np.issubdtype(data2.dtype, np.number):
        print(f"Error: Non-numeric data detected in {var_name} for {group1_name} or {group2_name}")
        print(f"{group1_name} dtype: {data1.dtype}, {group2_name} dtype: {data2.dtype}")
        print(f"{group1_name} sample: {data1.head()}")
        print(f"{group2_name} sample: {data2.head()}")
        return "Invalid", np.nan, np.nan

    # Check if data is empty after dropping NaNs
    if len(data1) == 0 or len(data2) == 0:
        print(f"Error: Empty dataset for {var_name} in {group1_name} or {group2_name} after dropping NaNs")
        return "Empty", np.nan, np.nan

    # Perform Shapiro-Wilk test for normality
    stat1, p1 = shapiro(data1)
    stat2, p2 = shapiro(data2)
    
    print(f"{var_name} - {group1_name} Shapiro-Wilk: p={p1:.4f}")
    print(f"{var_name} - {group2_name} Shapiro-Wilk: p={p2:.4f}")
    
    # Choose test based on normality
    if p1 > 0.05 and p2 > 0.05:  # Both are normal
        print(f"{var_name} - Using t-test (both groups normal)")
        stat, p = ttest_ind(data1, data2, equal_var=True)  # Assuming equal variances
        test_name = "t-test"
    else:
        print(f"{var_name} - Using Mann-Whitney U test (non-normal distribution)")
        stat, p = mannwhitneyu(data1, data2, alternative='two-sided')
        test_name = "Mann-Whitney U"
    
    return test_name, stat, p

group_ani = []
control_ids = ['65165', '65091', '63383', '66539', '65622']
exp_ids = ['65588', '63385', '66538', '66537', '66922']
for index, row in combined_df.iterrows():
    if str(row['matlab_animal']) in control_ids:
        group_ani.append("control")
    else:
        group_ani.append("exp")
combined_df['group_ani']=None
combined_df['group_ani']=group_ani


In [5]:
from scipy.stats import binomtest
control_df_deep = combined_df[(combined_df['group_ani'] == 'control') & (combined_df['depth'] == 'deep')]
exp_df_deep = combined_df[(combined_df['group_ani'] == 'exp') & (combined_df['depth'] == 'deep')]


# # Control_superficial vs. Exp_superficial
# control_superficial = combined_df[(combined_df['group_ani'] == 'control') & (combined_df['depth'] == 'superficial')][var]
# exp_superficial = combined_df[(combined_df['group_ani'] == 'exp') & (combined_df['depth'] == 'superficial')][var]

control_df_place = control_df_deep[(control_df_deep['h_0_place_cell'] == 1) & 
                                (control_df_deep['Information_content_rate'] >= 1.68) & 
                                (control_df_deep['matlab_maxfsize'] >= 20) ]
exp_df_place = exp_df_deep[(exp_df_deep['h_0_place_cell'] == 1) & 
                        (exp_df_deep['Information_content_rate'] >= 1.68) & 
                        (exp_df_deep['matlab_maxfsize'] >= 20) ]

# Calculate numbers for pie charts
control_place_deep = len(control_df_place)
control_non_place_deep = len(control_df_deep) - control_place_deep
exp_place_deep = len(exp_df_place)
exp_non_place_deep = len(exp_df_deep) - exp_place_deep
result = binomtest(exp_place_deep, len(exp_df_deep), p=(control_place_deep /len(control_df_deep)), alternative='less')
p_value_deep = result.pvalue
p_value_deep

0.12163618123000769

In [6]:
from scipy.stats import binomtest
control_df_superficial = combined_df[(combined_df['group_ani'] == 'control') & (combined_df['depth'] == 'superficial')]
exp_df_superficial = combined_df[(combined_df['group_ani'] == 'exp') & (combined_df['depth'] == 'superficial')]

# # Control_superficial vs. Exp_superficial
# control_superficial = combined_df[(combined_df['group_ani'] == 'control') & (combined_df['depth'] == 'superficial')][var]
# exp_superficial = combined_df[(combined_df['group_ani'] == 'exp') & (combined_df['depth'] == 'superficial')][var]

control_df_place_superficial = control_df_superficial[(control_df_superficial['h_0_place_cell'] == 1) & 
                                (control_df_superficial['Information_content_rate'] >= 1.68) & 
                                (control_df_superficial['matlab_maxfsize'] >= 20)]
exp_df_place_superficial = exp_df_superficial[(exp_df_superficial['h_0_place_cell'] == 1) & 
                        (exp_df_superficial['Information_content_rate'] >= 1.68) & 
                        (exp_df_superficial['matlab_maxfsize'] >= 20) ]

# Calculate numbers for pie charts
control_place_superficial = len(control_df_place_superficial)
control_non_place_superficial = len(control_df_superficial) - control_place_superficial
exp_place_superficial = len(exp_df_place_superficial)
exp_non_place_superficial = len(exp_df_superficial) - exp_place_superficial
result = binomtest(exp_place_superficial, len(exp_df_superficial), p=(control_place_superficial /len(control_df_superficial)), alternative='less')
p_value_superficial = result.pvalue
p_value_superficial

0.01564506099757105

In [48]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy.stats import ks_2samp

# Create figure with high DPI
fig = plt.figure(figsize=(7.2, 10), dpi=1200, constrained_layout=False)  # Switch to manual layout for fine control
plt.rcParams.update({'font.size': 8})  # Keep font size at 8 for consistency

# Define GridSpec with adjusted ratios
gs = gridspec.GridSpec(4, 3, height_ratios=[1, 1,1,1], width_ratios=[1, 1, 1])

# Adjust global padding parameters to prevent overlap
plt.rcParams.update({
    'axes.labelpad': 4,  # Increase label padding for better spacing
    'ytick.major.pad': 1,  # Increase y-tick padding
    'xtick.major.pad': 1,  # Increase x-tick padding
    'ytick.major.size': 2,
    'xtick.major.size': 2
})

linewidth = 0.5

# Define axes for the first row (deep)
ax4_1 = fig.add_subplot(gs[0, 0])
ax4_2 = fig.add_subplot(gs[0, 1])
ax4_3 = fig.add_subplot(gs[0, 2])
ax4_4 = fig.add_subplot(gs[1, 0])
ax4_5 = fig.add_subplot(gs[1, 1])
ax4_6 = fig.add_subplot(gs[1, 2])
axis_deep = [ax4_1, ax4_2, ax4_3, ax4_4, ax4_5, ax4_6]

# Define axes for the second row (superficial)
ax7_1 = fig.add_subplot(gs[2, 0])
ax7_2 = fig.add_subplot(gs[2, 1])
ax7_3 = fig.add_subplot(gs[2, 2])
ax7_4 = fig.add_subplot(gs[3, 0])
ax7_5 = fig.add_subplot(gs[3, 1])
ax7_6 = fig.add_subplot(gs[3, 2])
axis_superficial = [ax7_1, ax7_2, ax7_3, ax7_4, ax7_5, ax7_6]

# Define axis limits dictionary
xlim_dict = {
    'half_width': (None, 0.0003),
    'recovery_slope': (None, None),
    'repolarization_slope': None,
    'mean_firing_rate': (0, 8),
    'peak_trough_ratio': (None, None),
    'peak_to_valley': (None, 0.001),
    'bursting_index': (-1, 1),
    'mean_inter_spike_interval': (0.1, 8),
    'amplitude_median': None,
    'mode_inter_spike_interval': (0, 0.02)
}

palette = {'control': 'blue', 'exp': 'red'}

scale_transefers = [0.001, 1000, 1000, 1, 0.00001, 0.000001]
control_color = "blue"
exp_color = "red"

# Plot for deep layer
for i, (var, title) in enumerate(zip(variables, titles)):
    # Data for deep layer
    scale_transefers = [0.001, 1000, 1000, 1, 0.00001, 0.000001]
    control_deep = combined_df[(combined_df['group_ani'] == 'control') & (combined_df['depth'] == 'deep')][var]
    exp_deep = combined_df[(combined_df['group_ani'] == 'exp') & (combined_df['depth'] == 'deep')][var]
    control_values =control_deep.dropna()
    exp_values =exp_deep.dropna()

    # Statistical test for superficial
    test_name_sup, stat_sup, p_values = choose_stat_test(control_deep, exp_deep, title, "Control_superficial", "Exp_superficial")
    print(f"{title} - Control_superficial vs. Exp_superficial ({test_name_sup}): statistic={stat_sup:.3f}, p-value={p_values:.4f}")
    
    # KS test
    ks_stat_sup, p_values_s = ks_2samp(control_deep, exp_deep, nan_policy='omit')
    plot_df = pd.DataFrame({
            'value': pd.concat([control_values* scale_transefers[i], exp_values* scale_transefers[i]]),
            'group': ['Control'] * len(control_values) + ['Experimental'] * len(exp_values)
        })
    all_values = plot_df['value']
    mean_val = all_values.mean()
    std_val = all_values.std()
    plot_df_filtered = plot_df[(plot_df['value'] >= mean_val - 3 * std_val) & 
                                (plot_df['value'] <= mean_val + 3 * std_val)]

    sns.violinplot(
        data=plot_df_filtered, x='group', y='value', ax=axis_deep[i],inner = None,
        palette={"Control": control_color, "Experimental": exp_color}, width=0.8, cut=0, edgecolor=None
    )
    for patch in axis_superficial[i].collections:
        patch.set_alpha(1)
    sns.boxplot(
        
        data=plot_df_filtered, 
        x='group', y='value',
        palette={"Control": "black", "Experimental": "black"},
        width=0.3, 
        fill=False,  # No fill, only outlines
        showfliers=False,  # Hide outliers
        showmeans=False,  # Remove mean marker, assuming midline is the median
        linewidth=1.5,  # Makes the lines narrower (thinner)
        ax=axis_deep[i]  # Add this
    )

    # # Statistical test for deep
    test_name_deep, stat_deep, p_valued = choose_stat_test(control_deep, exp_deep, title, "Control_deep", "Exp_deep")
    # print(f"{title} - Control_deep vs. Exp_deep ({test_name_deep}): statistic={stat_deep:.3f}, p-value={p_valued:.4f}")
    
    # # KS test
    # ks_stat_deep, p_value_d = ks_2samp(control_deep, exp_deep, nan_policy='omit')

    # # Violin and swarm plots
    # sns.violinplot(
    #     data=combined_df[combined_df['depth'] == 'deep'], x='group_ani', y=var, ax=axis_deep[i],
    #     hue='group_ani', inner="quartiles", palette=palette, cut=0, edgecolor='white'
    # )
    # sns.swarmplot(
    #     data=combined_df[combined_df['depth'] == 'deep'], x='group_ani', y=var, ax=axis_deep[i], size=0.5,
    #     hue='group_ani', palette=palette, alpha=1, legend=False
    # )
    
    # Axis customization
    axis_deep[i].set_xlabel('')  # Remove x-label to reduce clutter
    axis_deep[i].set_ylabel(title, fontsize=8, labelpad=5)  # Set y-label with padding
    axis_deep[i].legend().set_visible(False)
    axis_deep[i].spines['top'].set_visible(False)
    axis_deep[i].spines['right'].set_visible(False)
    #axis_deep[i].set_ylim(xlim_dict[var])
    axis_deep[i].set_xticklabels(['CR;DTA-', 'CR;DTA+'], rotation=-45, ha='left', fontsize=7)  # Rotate and align labels
    
    # Add significance bar
    y_max = axis_deep[i].get_ylim()[1]
    bar_height = y_max * 0.1
    x_positions = [0, 1]
    p_val = p_valued
    if 0.01 < p_val < 0.05:
        axis_deep[i].plot(x_positions, [y_max + bar_height, y_max + bar_height], color='black', lw=1.5)
        axis_deep[i].text(0.5, y_max + bar_height * 1.1, '*', ha='center', va='bottom', fontsize=8)
    elif 0.001 < p_val <= 0.01:
        axis_deep[i].plot(x_positions, [y_max + bar_height, y_max + bar_height], color='black', lw=1.5)
        axis_deep[i].text(0.5, y_max + bar_height * 1.1, '**', ha='center', va='bottom', fontsize=8)
    elif p_val <= 0.001:
        axis_deep[i].plot(x_positions, [y_max + bar_height, y_max + bar_height], color='black', lw=1.5)
        axis_deep[i].text(0.5, y_max + bar_height * 1.1, '***', ha='center', va='bottom', fontsize=8)


# Plot for superficial layer
for i, (var, title) in enumerate(zip(variables, titles)):
    # Data for superficial layer
    control_superficial = combined_df[(combined_df['group_ani'] == 'control') & (combined_df['depth'] == 'superficial')][var]
    exp_superficial = combined_df[(combined_df['group_ani'] == 'exp') & (combined_df['depth'] == 'superficial')][var]

    control_values =control_superficial.reset_index(drop=True) * scale_transefers[i]
    exp_values =exp_superficial.reset_index(drop=True) * scale_transefers[i]

    # Statistical test for superficial
    test_name_sup, stat_sup, p_values = choose_stat_test(control_superficial, exp_superficial, title, "Control_superficial", "Exp_superficial")
    print(f"{title} - Control_superficial vs. Exp_superficial ({test_name_sup}): statistic={stat_sup:.3f}, p-value={p_values:.4f}")
    
    # KS test
    ks_stat_sup, p_values_s = ks_2samp(control_superficial, exp_superficial, nan_policy='omit')
    plot_df = pd.DataFrame({
            'value': pd.concat([control_values, exp_values]),
            'group': ['Control'] * len(control_values) + ['Experimental'] * len(exp_values)
        })
    all_values = plot_df['value']
    mean_val = all_values.mean()
    std_val = all_values.std()
    plot_df_filtered = plot_df[(plot_df['value'] >= mean_val - 3 * std_val) & 
                                (plot_df['value'] <= mean_val + 3 * std_val)]

    sns.violinplot(
        data=plot_df_filtered, x='group', y='value', ax=axis_superficial[i],inner = None,
        palette={"Control": control_color, "Experimental": exp_color}, width=0.8, cut=0, edgecolor=None
    )
    for patch in axis_superficial[i].collections:
        patch.set_alpha(1)
    sns.boxplot(
        data=plot_df_filtered, 
        x='group', y='value',
        palette={"Control": "black", "Experimental": "black"},
        width=0.3, 
        fill=False,  # No fill, only outlines
        showfliers=False,  # Hide outliers
        showmeans=False,  # Remove mean marker, assuming midline is the median
        linewidth=1.5,  # Makes the lines narrower (thinner)
        ax=axis_superficial[i]  # Add this
    )
    # # Violin and swarm plots
    # sns.violinplot(
    #     data=combined_df[combined_df['depth'] == 'superficial'], x='group_ani', y=var, ax=axis_superficial[i],
    #     hue='group_ani', inner="quartiles", palette=palette, cut=0, edgecolor='white'
    # )
    # sns.swarmplot(
    #     data=combined_df[combined_df['depth'] == 'superficial'], x='group_ani', y=var, ax=axis_superficial[i], size=0.5,
    #     hue='group_ani', palette=palette, alpha=1, legend=False
    # )
    
    # Axis customization
    axis_superficial[i].set_xlabel('')  # Remove x-label to reduce clutter
    axis_superficial[i].set_ylabel(title, fontsize=8, labelpad=5)  # Set y-label with padding
    axis_superficial[i].legend().set_visible(False)
    axis_superficial[i].spines['top'].set_visible(False)
    axis_superficial[i].spines['right'].set_visible(False)
    #axis_superficial[i].set_ylim(xlim_dict[var])
    axis_superficial[i].set_xticklabels(['CR;DTA-', 'CR;DTA+'], rotation=-45, ha='left', fontsize=7)  # Rotate and align labels
    
    # Add significance bar
    y_max = axis_superficial[i].get_ylim()[1]
    bar_height = y_max * 0.1
    x_positions = [0, 1]
    p_val = p_values
    if 0.01 < p_val < 0.05:
        axis_superficial[i].plot(x_positions, [y_max + bar_height, y_max + bar_height], color='black', lw=1.5)
        axis_superficial[i].text(0.5, y_max + bar_height * 1.1, '*', ha='center', va='bottom', fontsize=8)
    elif 0.001 < p_val <= 0.01:
        axis_superficial[i].plot(x_positions, [y_max + bar_height, y_max + bar_height], color='black', lw=1.5)
        axis_superficial[i].text(0.5, y_max + bar_height * 1.1, '**', ha='center', va='bottom', fontsize=8)
    elif p_val <= 0.001:
        axis_superficial[i].plot(x_positions, [y_max + bar_height, y_max + bar_height], color='black', lw=1.5)
        axis_superficial[i].text(0.5, y_max + bar_height * 1.1, '***', ha='center', va='bottom', fontsize=8)

# Adjust layout to prevent overlap
plt.subplots_adjust(top=0.95, bottom=0.15, left=0.1, right=0.95, hspace=0.4, wspace=0.3)  # Increased hspace and wspace
plt.savefig(r'Q:\sachuriga\CR_CA1_paper\Figures/suppfig5.png', transparent=True, dpi=1200, bbox_inches='tight')
plt.show()

C:\Users\sachur\AppData\Local\Temp\ipykernel_33580\4128129263.py:87: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.violinplot(
C:\Users\sachur\AppData\Local\Temp\ipykernel_33580\4128129263.py:93: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
C:\Users\sachur\AppData\Local\Temp\ipykernel_33580\4128129263.py:130: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  axis_deep[i].set_xticklabels(['CR;DTA-', 'CR;DTA+'], rotation=-45, ha='left', fontsize=7)  # Rotate and al

Amplitude median (mV) - Control_superficial Shapiro-Wilk: p=0.0000
Amplitude median (mV) - Exp_superficial Shapiro-Wilk: p=0.0000
Amplitude median (mV) - Using Mann-Whitney U test (non-normal distribution)
Amplitude median (mV) - Control_superficial vs. Exp_superficial (Mann-Whitney U): statistic=9842.000, p-value=0.4195
Amplitude median (mV) - Control_deep Shapiro-Wilk: p=0.0000
Amplitude median (mV) - Exp_deep Shapiro-Wilk: p=0.0000
Amplitude median (mV) - Using Mann-Whitney U test (non-normal distribution)
Half width (ms) - Control_superficial Shapiro-Wilk: p=0.0000
Half width (ms) - Exp_superficial Shapiro-Wilk: p=0.0000
Half width (ms) - Using Mann-Whitney U test (non-normal distribution)
Half width (ms) - Control_superficial vs. Exp_superficial (Mann-Whitney U): statistic=9542.500, p-value=0.7277
Half width (ms) - Control_deep Shapiro-Wilk: p=0.0000
Half width (ms) - Exp_deep Shapiro-Wilk: p=0.0000
Half width (ms) - Using Mann-Whitney U test (non-normal distribution)
Peak to vall

C:\Users\sachur\AppData\Local\Temp\ipykernel_33580\4128129263.py:130: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  axis_deep[i].set_xticklabels(['CR;DTA-', 'CR;DTA+'], rotation=-45, ha='left', fontsize=7)  # Rotate and align labels
C:\Users\sachur\AppData\Local\Temp\ipykernel_33580\4128129263.py:87: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.violinplot(
C:\Users\sachur\AppData\Local\Temp\ipykernel_33580\4128129263.py:93: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no

peak trough ratio - Control_deep Shapiro-Wilk: p=0.0000
peak trough ratio - Exp_deep Shapiro-Wilk: p=0.0000
peak trough ratio - Using Mann-Whitney U test (non-normal distribution)
Recovery slope - Control_superficial Shapiro-Wilk: p=0.0000
Recovery slope - Exp_superficial Shapiro-Wilk: p=0.0000
Recovery slope - Using Mann-Whitney U test (non-normal distribution)
Recovery slope - Control_superficial vs. Exp_superficial (Mann-Whitney U): statistic=9475.000, p-value=0.8068
Recovery slope - Control_deep Shapiro-Wilk: p=0.0000
Recovery slope - Exp_deep Shapiro-Wilk: p=0.0000
Recovery slope - Using Mann-Whitney U test (non-normal distribution)
repolarization_slope - Control_superficial Shapiro-Wilk: p=0.0000
repolarization_slope - Exp_superficial Shapiro-Wilk: p=0.0000
repolarization_slope - Using Mann-Whitney U test (non-normal distribution)
repolarization_slope - Control_superficial vs. Exp_superficial (Mann-Whitney U): statistic=9084.000, p-value=0.7238
repolarization_slope - Control_deep

C:\Users\sachur\AppData\Local\Temp\ipykernel_33580\4128129263.py:207: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  axis_superficial[i].set_xticklabels(['CR;DTA-', 'CR;DTA+'], rotation=-45, ha='left', fontsize=7)  # Rotate and align labels
C:\Users\sachur\AppData\Local\Temp\ipykernel_33580\4128129263.py:173: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.violinplot(
C:\Users\sachur\AppData\Local\Temp\ipykernel_33580\4128129263.py:179: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is calle

Half width (ms) - Control_superficial Shapiro-Wilk: p=0.0071
Half width (ms) - Exp_superficial Shapiro-Wilk: p=0.0007
Half width (ms) - Using Mann-Whitney U test (non-normal distribution)
Half width (ms) - Control_superficial vs. Exp_superficial (Mann-Whitney U): statistic=4577.000, p-value=0.0009
Peak to valley (ms) - Control_superficial Shapiro-Wilk: p=0.1609
Peak to valley (ms) - Exp_superficial Shapiro-Wilk: p=0.0000
Peak to valley (ms) - Using Mann-Whitney U test (non-normal distribution)
Peak to valley (ms) - Control_superficial vs. Exp_superficial (Mann-Whitney U): statistic=4537.000, p-value=0.0014
peak trough ratio - Control_superficial Shapiro-Wilk: p=0.0274
peak trough ratio - Exp_superficial Shapiro-Wilk: p=0.0062
peak trough ratio - Using Mann-Whitney U test (non-normal distribution)
peak trough ratio - Control_superficial vs. Exp_superficial (Mann-Whitney U): statistic=1817.000, p-value=0.0000


C:\Users\sachur\AppData\Local\Temp\ipykernel_33580\4128129263.py:207: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  axis_superficial[i].set_xticklabels(['CR;DTA-', 'CR;DTA+'], rotation=-45, ha='left', fontsize=7)  # Rotate and align labels
C:\Users\sachur\AppData\Local\Temp\ipykernel_33580\4128129263.py:173: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.violinplot(
C:\Users\sachur\AppData\Local\Temp\ipykernel_33580\4128129263.py:179: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is calle

Recovery slope - Control_superficial Shapiro-Wilk: p=0.0000
Recovery slope - Exp_superficial Shapiro-Wilk: p=0.1867
Recovery slope - Using Mann-Whitney U test (non-normal distribution)
Recovery slope - Control_superficial vs. Exp_superficial (Mann-Whitney U): statistic=2551.000, p-value=0.0030
repolarization_slope - Control_superficial Shapiro-Wilk: p=0.0000
repolarization_slope - Exp_superficial Shapiro-Wilk: p=0.0000
repolarization_slope - Using Mann-Whitney U test (non-normal distribution)
repolarization_slope - Control_superficial vs. Exp_superficial (Mann-Whitney U): statistic=3187.000, p-value=0.3198


In [30]:


control_deep = combined_df[(combined_df['group_ani'] == 'control') & (combined_df['depth'] == 'deep')]['peak_to_valley']
exp_deep = combined_df[(combined_df['group_ani'] == 'exp') & (combined_df['depth'] == 'deep')]['peak_to_valley']
control_values =  control_deep.dropna().multiply(scale_transefers[i])
exp_values = exp_deep.dropna().multiply(scale_transefers[i])

In [40]:
control_superficial.values

array([ 830949.77578961, 1097191.15977699,  898118.9314514 ,
        613033.0265153 , 1163382.95411271,  793261.16868012,
        883563.37878221,  562171.14966806, 1218080.61714341,
        717109.75163688, 1005561.73697817, 1741139.61402897,
       1068829.31149722,  934334.36710567,  905345.14084458,
        689378.4610922 ,  904600.99075175,  470976.27433962,
       1001211.64197757,  981640.25605043, 1089532.04455864,
       1776763.28925057, 1020090.64050037,  878937.36757932,
       1176180.78308019,  874661.56358662,  918116.99361044,
       1041500.92020151, 1768544.18640673,  563765.66219577,
        519606.76818151, 1528925.80895177,  831666.25594723,
        707469.63769193, 1164675.67321385,  435043.42587418,
        740646.70570145, 1246951.15086848,  928104.48197264,
       1634050.96329079,  830229.534618  ,  719113.98078823,
        251914.67868111, 1030775.8172672 , 1437756.11085742,
       1303079.724079  , 1517203.71270016, 1023433.85217356,
        948520.98948804,